# iris Classification

## libraries

In [53]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

## Data Gathering

In [54]:
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
#cat={'Iris-setosa':0,'Iris-versicolor':1,'Iris-virginica':2}
a=pd.read_csv("data/iris.csv", names=names)
a = shuffle(a)
#a['class']=[cat[i] for i in a['class']]
dummies=pd.get_dummies(a)

array=dummies.values
x=array[:,:4]
y=array[:,4:]
validation_size=0.10
seed=7
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=validation_size, random_state=seed)

## Building Neural network

In [50]:
class Neural_Network(object):
  def __init__(self):
  #parameters
    self.inputSize = 4
    self.hl1Size = 10
    self.hl2Size = 10
    self.outputSize = 3

  #weights
    self.W1 = np.random.randn(self.inputSize, self.hl1Size) # (4*10) weight matrix from input to hidden layer 1
    self.W2 = np.random.randn(self.hl1Size,self.hl2Size) #(10*10) weight matrix for hidden layer 1 to hidden layer 2
    self.W3 = np.random.randn(self.hl2Size, self.outputSize) # (3x1) weight matrix from hidden layer 2 to output layer
    self.b1=np.random.randn(self.hl1Size)
    self.b2=np.random.randn(self.hl2Size)
    self.b3=np.random.randn(self.outputSize)

  def forward(self, X):
    #forward propagation through our network
    self.z = np.dot(X, self.W1)+self.b1 # dot product of X (input) and first set of weights and adding bias
    self.hl1 = self.sigmoid(self.z) # activation function
    
    self.z2 = np.dot(self.hl1, self.W2)+self.b2 # dot product of hidden layer1 and second set of weights and adding bias
    self.hl2 = self.sigmoid(self.z2) # activation function
    
    self.z3 = np.dot(self.hl2, self.W3)+self.b3 # dot product of hidden layer2 and third set of weights and adding bias
    o = self.sigmoid(self.z3) # final activation function
    
    return o

  def sigmoid(self, s):
    # activation function
    return 1/(1+np.exp(-s))

  def sigmoidPrime(self, s):
    #derivative of sigmoid
    return s * (1 - s)

  def backward(self, X, y, o):
    # backward propagate through the network
    self.o_error = y - o # error in output
    self.o_delta = self.o_error*self.sigmoidPrime(o)

    self.z3_error = self.o_delta.dot(self.W3.T)
    self.z3_delta = self.z3_error*self.sigmoidPrime(self.hl2) 
    
    self.z2_error = self.z3_delta.dot(self.W2.T) 
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.hl1) 

    self.W1 += X.reshape(4,1).dot(self.z2_delta.reshape(1,10)) # adjusting first set (input --> hidden) weights
    self.W2 += self.hl1.T.dot(self.z3_delta) # adjusting second set (hidden --> output) weights
    self.W3 += self.hl2.reshape(10,1).dot(self.o_delta.reshape(1,3)) # adjusting second set (hidden --> output) weights

  def train(self, X, y):
    min,hw1,hw2,hw3,hb1,hb2,hb3=1,0,0,0,0,0,0
    print("Before Training loss: "+str(np.mean(np.square(y - self.forward(X)))))
    for j in range(30):
        avg_loss=[]
        for i in range(len(X)):
            o = self.forward(X[i])
            self.backward(X[i], y[i], o)
            l=np.mean(np.square(y - self.forward(X)))
            avg_loss.append(l)
            if min>l:
                min=l
                hw1,hb1=self.W1,self.b1
                hw2,hb2=self.W2,self.b2
                hw3,hb3=self.W3,self.b3
        print("avg loss of "+str(j)+" loop is " + str(np.mean(avg_loss)))
              
    print("min loss: "+str(min))
    self.W1,self.b1=hw1,hb1
    self.W2,self.b2=hw2,hb2
    self.W3,self.b3=hw3,hb3
    print("After Training loss: "+str(np.mean(np.square(y - self.forward(X)))))
    

  def predict(self,X):
    return self.forward(X).round()



In [51]:
NN = Neural_Network()
NN.train(X_train,Y_train)

Before Training loss: 0.24948435252524534
avg loss of 0 loop is 0.16971047157284264
avg loss of 1 loop is 0.13964193646437836
avg loss of 2 loop is 0.13138646170502263
avg loss of 3 loop is 0.1290181026645146
avg loss of 4 loop is 0.12784096105800632
avg loss of 5 loop is 0.12456899653122225
avg loss of 6 loop is 0.11966560567282171
avg loss of 7 loop is 0.11549308202738806
avg loss of 8 loop is 0.118438786129305
avg loss of 9 loop is 0.10621433404512785
avg loss of 10 loop is 0.1119075112944453
avg loss of 11 loop is 0.10475557231947003
avg loss of 12 loop is 0.0990661272518813
avg loss of 13 loop is 0.08887351755464885
avg loss of 14 loop is 0.10523635903049532
avg loss of 15 loop is 0.08808216363760452
avg loss of 16 loop is 0.0802398692199703
avg loss of 17 loop is 0.07923843588575126
avg loss of 18 loop is 0.07170862499373314
avg loss of 19 loop is 0.07192554013069294
avg loss of 20 loop is 0.06809234483290084
avg loss of 21 loop is 0.0738464807890234
avg loss of 22 loop is 0.0794

In [52]:
predictions=NN.predict(X_test)
print(accuracy_score(Y_test, predictions))

0.9333333333333333
